In [1]:
from rdkit import Chem
import psycopg2 as ps

In [2]:
c = ps.connect("dbname='drugbank' user='script' host='localhost' password=''")

In [3]:
cursor = c.cursor()

In [4]:
cursor.execute("select version()")
reply = cursor.fetchall()

In [5]:
print(reply)

[('PostgreSQL 9.5.10 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 4.4.7 20120313 (Red Hat 4.4.7-18), 64-bit',)]


In [6]:
cursor.execute("drop schema if exists drugbank cascade")
cursor.execute("create schema drugbank")

In [7]:
cursor.execute("""
create table drugbank.drugbank(
       id serial primary key,
       drugbank_id text,
       smiles text,
       molinstance mol,
       morganfp bfp,
       hba integer,
       hbd integer,
       mw float,
       logp float
);
""")

In [8]:
cursor.execute("select * from drugbank.drugbank")
print(cursor.fetchall())

[]


In [9]:
supp = Chem.SDMolSupplier("../data/drugbank.sdf")
drugbank = [mol for mol in supp if mol]

In [10]:
query = "insert into drugbank.drugbank (smiles, drugbank_id) values %s" \
            % ((', '.join(["('%s','%s')" % (Chem.MolToSmiles(mol), mol.GetProp('DRUGBANK_ID')) 
                         for mol in drugbank])))
cursor.execute(query)
print(query[:1000])

insert into drugbank.drugbank (smiles, drugbank_id) values ('CCC(C)C(NC(=O)C(CCC(=O)O)NC(=O)C(CCC(=O)O)NC(=O)C(Cc1ccccc1)NC(=O)C(CC(=O)O)NC(=O)CNC(=O)C(CC(N)=O)NC(=O)CNC(=O)CNC(=O)CNC(=O)CNC(=O)C1CCCN1C(=O)C(CCCNC(=N)N)NC(=O)C1CCCN1C(=O)C(N)Cc1ccccc1)C(=O)N1CCCC1C(=O)NC(CCC(=O)O)C(=O)NC(CCC(=O)O)C(=O)NC(Cc1ccc(O)cc1)C(=O)NC(CC(C)C)C(=O)O','DB00006'), ('CC(C)CC(NC(=O)C(COC(C)(C)C)NC(=O)C(Cc1ccc(O)cc1)NC(=O)C(CO)NC(=O)C(Cc1c[nH]c2ccccc12)NC(=O)C(Cc1cnc[nH]1)NC(=O)C1CCC(=O)N1)C(=O)NC(CCCN=C(N)N)C(=O)N1CCCC1C(=O)NNC(N)=O','DB00014'), ('N=C(N)NCCCC(NC(=O)C1CCCN1C(=O)C1CSSCCC(=O)NC(Cc2ccc(O)cc2)C(=O)NC(Cc2ccccc2)C(=O)NC(CCC(N)=O)C(=O)NC(CC(N)=O)C(=O)N1)C(=O)NCC(N)=O','DB00035'), ('CC(=O)NC(Cc1ccc2ccccc2c1)C(=O)NC(Cc1ccc(Cl)cc1)C(=O)NC(Cc1cccnc1)C(=O)NC(CO)C(=O)NC(Cc1ccc(O)cc1)C(=O)NC(CCCNC(N)=O)C(=O)NC(CC(C)C)C(=O)NC(CCCNC(=N)N)C(=O)N1CCCC1C(=O)NC(C)C(N)=O','DB00050'), ('CC=CCC(C)C(O)C1C(=O)NC(CC)C(=O)N(C)CC(=O)N(C)C(CC(C)C)C(=O)NC(C(C)C)C(=O)N(C)C(CC(C)C)C(=O)NC(C)C(=O)NC(C)C(=O)N(C)C(CC(C)

In [11]:
cursor.execute("select count(*) from drugbank.drugbank")
print(cursor.fetchall(), len(drugbank))

[(7110,)] 7110


In [12]:
cursor.execute("select * from drugbank.drugbank limit 10")

In [13]:
print(cursor.fetchall())

[(1, 'DB00006', 'CCC(C)C(NC(=O)C(CCC(=O)O)NC(=O)C(CCC(=O)O)NC(=O)C(Cc1ccccc1)NC(=O)C(CC(=O)O)NC(=O)CNC(=O)C(CC(N)=O)NC(=O)CNC(=O)CNC(=O)CNC(=O)CNC(=O)C1CCCN1C(=O)C(CCCNC(=N)N)NC(=O)C1CCCN1C(=O)C(N)Cc1ccccc1)C(=O)N1CCCC1C(=O)NC(CCC(=O)O)C(=O)NC(CCC(=O)O)C(=O)NC(Cc1ccc(O)cc1)C(=O)NC(CC(C)C)C(=O)O', None, None, None, None, None, None), (2, 'DB00014', 'CC(C)CC(NC(=O)C(COC(C)(C)C)NC(=O)C(Cc1ccc(O)cc1)NC(=O)C(CO)NC(=O)C(Cc1c[nH]c2ccccc12)NC(=O)C(Cc1cnc[nH]1)NC(=O)C1CCC(=O)N1)C(=O)NC(CCCN=C(N)N)C(=O)N1CCCC1C(=O)NNC(N)=O', None, None, None, None, None, None), (3, 'DB00035', 'N=C(N)NCCCC(NC(=O)C1CCCN1C(=O)C1CSSCCC(=O)NC(Cc2ccc(O)cc2)C(=O)NC(Cc2ccccc2)C(=O)NC(CCC(N)=O)C(=O)NC(CC(N)=O)C(=O)N1)C(=O)NCC(N)=O', None, None, None, None, None, None), (4, 'DB00050', 'CC(=O)NC(Cc1ccc2ccccc2c1)C(=O)NC(Cc1ccc(Cl)cc1)C(=O)NC(Cc1cccnc1)C(=O)NC(CO)C(=O)NC(Cc1ccc(O)cc1)C(=O)NC(CCCNC(N)=O)C(=O)NC(CC(C)C)C(=O)NC(CCCNC(=N)N)C(=O)N1CCCC1C(=O)NC(C)C(N)=O', None, None, None, None, None, None), (5, 'DB00091', 'CC=CCC

In [16]:
cursor.execute("select * from drugbank.drugbank where id > 500  and id < 600 limit 10;")

In [17]:
print(cursor.fetchall())

[(501, 'DB00615', 'COC1C=COC2(C)Oc3c(C)c(O)c4c(c3C2=O)C2=NC3(CCN(CC(C)C)CC3)NC2=C(NC(=O)C(C)=CC=CC(C)C(O)C(C)C(O)C(C)C(OC(C)=O)C1C)C4=O', None, None, None, None, None, None), (502, 'DB00616', 'COCCOCC(CC1(C(=O)NC2CCC(C(=O)O)CC2)CCCC1)C(=O)Oc1ccc2c(c1)CCC2', None, None, None, None, None, None), (503, 'DB00617', 'CCC1(C)OC(=O)N(C)C1=O', None, None, None, None, None, None), (504, 'DB00618', 'CN(C)C1C(O)=C(C(N)=O)C(=O)C2(O)C(O)=C3C(=O)c4c(O)ccc(Cl)c4C(O)C3CC12', None, None, None, None, None, None), (505, 'DB00619', 'Cc1ccc(NC(=O)c2ccc(CN3CCN(C)CC3)cc2)cc1Nc1nccc(-c2cccnc2)n1', None, None, None, None, None, None), (506, 'DB00620', 'CC12C=CC(=O)C=C1CCC1C3CC(O)C(O)(C(=O)CO)C3(C)CC(O)C12F', None, None, None, None, None, None), (507, 'DB00621', 'CC12COC(=O)CC1CCC1C2CCC2(C)C1CCC2(C)O', None, None, None, None, None, None), (508, 'DB00622', 'COC(=O)C1=C(C)NC(C)=C(C(=O)OCCN(C)Cc2ccccc2)C1c1cccc([N+](=O)[O-])c1', None, None, None, None, None, None), (509, 'DB00623', 'OCCN1CCN(CCCN2c3ccccc3Sc3ccc(C(F

In [18]:
cursor.execute("update drugbank.drugbank set molinstance = mol_from_smiles(smiles::cstring)")

In [20]:
cursor.execute("select * from drugbank.drugbank limit 5")

In [21]:
print(cursor.fetchall())

[(1, 'DB00006', 'CCC(C)C(NC(=O)C(CCC(=O)O)NC(=O)C(CCC(=O)O)NC(=O)C(Cc1ccccc1)NC(=O)C(CC(=O)O)NC(=O)CNC(=O)C(CC(N)=O)NC(=O)CNC(=O)CNC(=O)CNC(=O)CNC(=O)C1CCCN1C(=O)C(CCCNC(=N)N)NC(=O)C1CCCN1C(=O)C(N)Cc1ccccc1)C(=O)N1CCCC1C(=O)NC(CCC(=O)O)C(=O)NC(CCC(=O)O)C(=O)NC(Cc1ccc(O)cc1)C(=O)NC(CC(C)C)C(=O)O', 'CCC(C)C(NC(=O)C(CCC(=O)O)NC(=O)C(CCC(=O)O)NC(=O)C(Cc1ccccc1)NC(=O)C(CC(=O)O)NC(=O)CNC(=O)C(CC(N)=O)NC(=O)CNC(=O)CNC(=O)CNC(=O)CNC(=O)C1CCCN1C(=O)C(CCCNC(=N)N)NC(=O)C1CCCN1C(=O)C(N)Cc1ccccc1)C(=O)N1CCCC1C(=O)NC(CCC(=O)O)C(=O)NC(CCC(=O)O)C(=O)NC(Cc1ccc(O)cc1)C(=O)NC(CC(C)C)C(=O)O', None, None, None, None, None), (2, 'DB00014', 'CC(C)CC(NC(=O)C(COC(C)(C)C)NC(=O)C(Cc1ccc(O)cc1)NC(=O)C(CO)NC(=O)C(Cc1c[nH]c2ccccc12)NC(=O)C(Cc1cnc[nH]1)NC(=O)C1CCC(=O)N1)C(=O)NC(CCCN=C(N)N)C(=O)N1CCCC1C(=O)NNC(N)=O', 'CC(C)CC(NC(=O)C(COC(C)(C)C)NC(=O)C(Cc1ccc(O)cc1)NC(=O)C(CO)NC(=O)C(Cc1c[nH]c2ccccc12)NC(=O)C(Cc1cnc[nH]1)NC(=O)C1CCC(=O)N1)C(=O)NC(CCCN=C(N)N)C(=O)N1CCCC1C(=O)NNC(N)=O', None, None, None, None, None), (

In [22]:
cursor.execute("""
    update drugbank.drugbank set (hba, hbd, mw, logp) = 
    (mol_hba(molinstance), mol_hbd(molinstance), mol_amw(molinstance), mol_logp(molinstance))
""")
cursor.execute("select * from drugbank.drugbank limit 5")
print(cursor.fetchall())

[(3170, 'DB03587', 'CC(=O)C=O', 'CC(=O)C=O', None, 2, 0, 72.0630035400391, -0.225700005888939), (32, 'DB00139', 'O=C(O)CCC(=O)O', 'O=C(O)CCC(=O)O', None, 4, 2, 118.087997436523, -0.0641999989748001), (3956, 'DB04454', 'CCC(N)C(=O)O', 'CCC(N)C(=O)O', None, 3, 3, 103.121002197266, -0.191699996590614), (3957, 'DB04455', 'C[N+](C)(C)CC(=O)O', 'C[N+](C)(C)CC(=O)O', None, 3, 1, 118.15599822998, -0.222800001502037), (850, 'DB00971', 'S=[Se]=S', 'S=[Se]=S', None, 0, 0, 143.093994140625, 0.915600001811981)]


In [23]:
cursor.execute("update drugbank.drugbank set morganfp = morganbv_fp(molinstance)")
cursor.execute("select *  from drugbank.drugbank limit 5")
print(cursor.fetchall())

[(323, 'DB00435', '[N]=O', '[N]=O', '\\x00000000000000000000000000000000000400000000000000000000000000000000000000000040000000000001000000000000000000000000000000000000', 2, 0, 30.0060005187988, -0.447100013494492), (401, 'DB00515', 'N[Pt](N)(Cl)Cl', 'N[Pt](N)(Cl)Cl', '\\x00000000001000000001000000000000000008000010000000000000000000008000000000000000000000000000000000000000000000000000000000000000', 2, 4, 298.029998779297, 0.195299997925758), (646, 'DB00761', '[Cl-].[K+]', '[Cl-].[K+]', '\\x00000000000000000000000000000000000000000000000000000000000000000000000000000000000408000000000000000000000000000000000000000000', 0, 0, 74.5510025024414, -5.99200010299683), (1040, 'DB01164', '[Ca+2].[Cl-].[Cl-]', '[Ca+2].[Cl-].[Cl-]', '\\x00000000000000000000000000000000000000000000000000000000000000000000001000000000000400000000000000000000000000000000000000000000', 0, 0, 110.984001159668, -6.37279987335205), (1592, 'DB01828', 'CN', 'CN', '\\x00000000020000000000000000000000000008000000000000000

In [24]:
c.commit()